----

# CMSE401 Quiz Instructions

This quiz is designed to take approximately 30 minutes to complete (you will be given the entire class period).  

Please read the following instructions before starting the quiz.


> This is an open Internet quiz.  Feel free to use anything on the Internet with one important exception...
> 
> - **DO NOT** communicate live with other people or AI tools during the quiz (either verbally or online).  The goal here is to find answers to problems as you would in the real world and demonstrate your own ability to solve problems.  
> 
> You will be given **until the end of class** to complete this quiz.  Use your time wisely. 
> 
> **HINTS:**
> - Neatness and grammar are important.  We will ignore all notes or code we can not read or understand.
> - Read the entire quiz from beginning to end before starting.  Not all questions are equal in **points vs. time** so plan your time accordingly. 
> - Spaces for answers are provided. Delete the prompting text such as "Put your answer to the above question here" and replace it with your answer. Do not leave the prompting text with your answer.
> - Do not assume that the answer must be in the same format of the cell provided. Feel free to change the cell formatting (e.g., markdown to code, and vice versa) or add additional cells as needed to provide your answer.
> - When a question asks for an answer "**in your own words**" it is still okay to search the Internet for the answer as a reminder. *However*, we would like you to do more than cut and paste.  Make the answer your own. 
> - If you get stuck, try not to leave an answer blank. It is better to include some notes or stub functions so we have an idea about your thinking process so we can give you partial credit.   
> - Always provid links to any references you find helpful. 
> - Feel free to delete the provided check marks (&#9989;) as a way to keep track of which questions you have successfully completed. 

> **Honor Code**
> 
> I, agree to neither give nor receive any help on this quiz from other people.  I also understand that providing answers to questions on this quiz to other students is also an academic misconduct violation as is live communication or receiving answers to questions on this quiz from other people. It is important to me to be a person of integrity and that means that ALL ANSWERS on this quiz are my answers.
> 
> &#9989; **<font color=red>DO THIS:</font>** Include your name in the line below to acknowledge the above statement:

Trenton Smiley

---

# Moving Average Code

The following serial code computes a 1000 step moving average across the 10 million timestep series. The "series" array is initialized using a random walk and then the moving average is computed and stored into the "avg" array. Finally, the last value from both the "series" array and the "avg" array are printed so that the result can be verified. 

```c
#include "stdio.h"
#include <math.h>
#include <stdlib.h>
static long num_steps = 10000000;

int main(){
    int steps = num_steps;
    unsigned int seed = 1;
    int range=1000;
    double series[num_steps];
    double avg[num_steps-range];


    //Initialize values in list
    series[0]=10.0;
    for (int i=1;i<steps;i++) {
        series[i]=series[i-1]+ ((double) rand_r(&seed))/RAND_MAX-0.5;
    }
    for (int i=0; i<steps-range;i++){
        avg[i]=0;
    }


    //Compute averages
    for (int i=0; i<steps-range; i++){
        for (int j=0; j<=range;j++){
            avg[i]+=series[i+j];
        }
        avg[i]/=(double)range + 1.0;
    }

    //Print elements for comparison
    printf("%f %f\n\n",series[steps-1],avg[steps-range-1]);
}
```

&#9989; **<font color=red>Question 1</font>**: (10 points) First, log into a dev node on the HPCC. Copy the serial code into a C file and then compile and run the file to test it out. Write the code below that you used to compile and run the C code. 

```bash

gcc -o quiz3_serial quiz3_serial.c
./quiz3_serial


&#9989; **<font color=red>Question 2</font>**: (10 points) Now that you have the C code running on the HPCC, we want to get a baseline measure of performance. Run the code several times using the default `num_steps` and record the average runtime. Include both the average runtime and code used to measure the runtime below. 

```bash
#!/bin/bash
TOTAL_TIME=0
for i in {1..10}; do
    TIME_OUTPUT=$( { time -p ./a.out; } 2>&1 )
    REAL_TIME=$(echo "$TIME_OUTPUT" | grep real | awk '{print $2}')
    TOTAL_TIME=$(echo "$TOTAL_TIME + $REAL_TIME" | bc -l)
done

AVERAGE_TIME=$(echo "$TOTAL_TIME / 10" | bc -l)
echo "Average Execution Time: $AVERAGE_TIME seconds"
```

- Runtime: 27.892 seconds


&#9989; **<font color=red>Question 3</font>**: (20 points) Now we want to parallelize the code using CUDA. Copy the supplied serial code into a new CUDA file and modify/optimize the code so that it can leverage multiple threads. Paste your CUDA code below once complete. Be sure to compile the code before moving on. 

Here are some hints that may be useful when writing your code: 

- your kernel will need access to the 'avg' and 'series' vectors as well as the 'range' variable
- the inner loop can persist in your CUDA kernel
- use the CUDA_CALL function to help debug if you are running into issues (supplied below)

```c
#define CUDA_CALL(x) {cudaError_t cuda_error__ = (x); if (cuda_error__) printf("CUDA error: " #x " returned \"%s\"\n", cudaGetErrorString(cuda_error__));}
```


```c
#include "stdio.h"
#include <math.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#define BLOCK_SIZE 256

static long num_steps = 10000000;

__global__ void kernel(const double* series, double* avg, int steps, int range) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < steps - range) {
        double sum = 0.0;
        for (int j = 0; j <= range; j++) {
            sum += series[i + j];
        }
        avg[i] = sum / (double)(range + 1);
    }
}

int main() {
    int steps = num_steps;
    unsigned int seed = 1;
    int range=1000;
    double series[num_steps];
    double avg[num_steps-range];
    
    series[0]=10.0;
    for (int i=1;i<steps;i++) {
        series[i]=series[i-1]+ ((double) rand_r(&seed))/RAND_MAX-0.5;
    }
    for (int i=0; i<steps-range;i++){
        avg[i]=0;
    }

    double *d_series, *d_avg;

    cudaMalloc((void**)&d_series, steps * sizeof(double));
    cudaMalloc((void**)&d_avg, (steps - range) * sizeof(double));

    cudaMemcpy(d_series, series, steps * sizeof(double), cudaMemcpyHostToDevice);

    int gridSize = (steps - range + BLOCK_SIZE - 1) / BLOCK_SIZE;
    kernel<<<gridSize, BLOCK_SIZE>>>(d_series, d_avg, steps, range);
    
    cudaMemcpy(avg, d_avg, (steps - range) * sizeof(double), cudaMemcpyDeviceToHost);

    cudaFree(d_series);
    cudaFree(d_avg);

    printf("%f %f\n\n",series[steps-1],avg[steps-range-1]);

    return 0;
}
```

&#9989; **<font color=red>Question 4</font>**: (10 points) Now lets setup a SLURM submission script. In this job script, do the following:

- set the allocated time to 5 minutes
- set the number of nodes to 1
- set the number of cores to 1
- set the memory to 1 Gb
- set the job name to Quiz3
- set the gpus to v100 with 1 gpu
- finally, include the code used to run and benchmark the parallel moving average code

Once completed, copy and paste the contents of the submission script below. 


```batch
#!/bin/bash
#SBATCH --time=00:05:00
#SBATCH --nodes=1
#SBATCH --cpus-per-task=1
#SBATCH --job-name=Quiz3
#SBATCH --gpus-per-node=v100:1
#SBATCH --output=quiz3_submission.txt
/usr/bin/time -v ./quiz3_cuda &> quiz3_submission.txt
```



&#9989; **<font color=red>Question 5</font>**: (10 points) Now lets benchmark our CUDA code using the submission script from the previous question. Submit the job several times and then record the average runtime. Record the code used to submit the jobs to the HPCC. 

```bash
#!/bin/bash
for i in {1..10}; do
    sbatch quiz3_submission.sh
done
```

- Average runtime: .29 seconds

&#9989; **<font color=red>Question 6</font>**: (10 points) Did your code run faster or slower than the serial version? If it ran faster, explain what design choices you made to ensure it would run faster. If it ran slower, explain why you think this is the case.

The code had about a 100 times speedup with the Cuda implementation. In order to ensure the speed up I parallelized the most memory and time intensive section of the code which was the outer loop in calculating the moving averages. In doing this, each thread block was responsible for only computing a fraction of the total time steps instead of running through a very large number of time steps sequentially. 

&#9989; **<font color=red>Question 7</font>**: (10 points) Describe how tiling could be applied to the code. You do not need to implement it, just describe at an overview level what tiling is and how tiling might be applied to this specific problem. 

Tiling could be applied by giving each block a local copy of the series and avg variables as they are accessed repeatedly throughout the loop and this would allow them to have a closer memory location for quicker access and retrieval

# Congratulations

You are done with your quiz. Please save the file and upload the jupyter notebook and any other necessary files to the D2L dropbox. 

Written by Dr. Nathan Haut, Michigan State University
<a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.

----